### Scraping  Toronto postal code and neighborhood

In [5]:
#importing key libraries

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup


In [6]:
#scraping the table and create a dataframe


df = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050')[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Cleaning the scraped table

In [8]:
# we exclude the Not assigned rows

df = df[df.Borough!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [16]:
#we now join these two rows will be combined into one row with the neighborhoods separated with a comma.

df = df.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))

In [17]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
df.shape

(103, 3)

### Now we get the Geospatial data with the link provided by coursera assignment

In [37]:
df_geosp = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

df_geosp.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:


df_geosp.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [39]:
#now we merge the 2 dataframes df and df_geosp

df = pd.merge(df, df_geosp, left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1)

df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Now we start to get coordinates for Toronto using Geopy and then we draw the map with Folium

In [42]:
# coordinates of Toronto

geolocator = Nominatim(user_agent="IBM Coursera")
location = geolocator.geocode("Toronto")
latitude = location.latitude
longitude = location.longitude

print('Coordinates of Toronto are latitude:', latitude, 'longitude:', longitude)

Coordinates of Toronto are latitude: 43.6534817 longitude: -79.3839347


In [43]:
# now we draw the map of Toronto with Folium and we make some markers of the neighbours

import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, borough, postcode, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Postcode'], df['Neighbourhood']):
  label = '{}, {}'.format(neighbourhood, borough)
  label=folium.Popup(label)
  folium.CircleMarker(
      [lat,lng],
      radius=8,
      color='blue',
      popup=label,
      fill_color='#3186cc',
      fill=True,
      fill_opacity=0.7

  ).add_to(map_toronto)

map_toronto



### Segment and cluster only the neighborhoods in Downtown Toronto

In [44]:
downtown_data = df[df['Borough']=='Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [45]:
# now we get the coordinates of DOwntown Toronto with geopy again

geolocator = Nominatim(user_agent="IBM Coursera")
location = geolocator.geocode("Downtown Toronto	")
latitude = location.latitude
longitude = location.longitude

print('Coordinates of Downtown Toronto are {} and {}'.format(latitude, longitude))

Coordinates of Downtown Toronto are 43.6541737 and -79.38081162653639


### Now we draw the downtown map with Folium

In [46]:
map_downtown = folium.Map(location=[latitude,longitude], zoom_start=15)

for lat, lng, postcode, borough, neighbourhood in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Postcode'], downtown_data['Borough'], downtown_data['Neighbourhood']):
  label = '{}, {}'.format(neighbourhood, borough)
  label = folium.Popup(label)
  folium.CircleMarker(
      [lat,lng],
      radius=8,
      color='blue',
      fill_color='#3186cc',
      fill_opacity=0.7,
      popup=label

  ).add_to(map_downtown)

map_downtown

### Now we explore the first neighbourhood of Downtown Toronto: Rosedale

In [47]:
neighbourhood_name = downtown_data.loc[0, 'Neighbourhood']
neighbourhood_lat = downtown_data.loc[0, 'Latitude']
neighbourhood_lng = downtown_data.loc[0, 'Longitude']

print("The geographical coordinates of {} are {}, {}".format(neighbourhood_name, neighbourhood_lat, neighbourhood_lng))

The geographical coordinates of Rosedale are 43.6795626, -79.37752940000001


### Now we retrieve the 100 venues in ROsdale using Foursquare Api

In [49]:
# we will first start putting my personal FOursquare credentials, radius, limit, lat and long, etc.

LIMIT = 100
radius = 500
CLIENT_ID = 'Z5R30YV2S44XZOKFOBIH4NOBKDNK50UM5JEUGWC51TBFGWKY'
CLIENT_SECRET = 'QSHHNLLS50BFLHHLQX2PZP3LZFAMCZ401MLBXDZUNR24FHVE'
VERSION = '20210528'
neighbourhood_lat, neighbourhood_lng


(43.6795626, -79.37752940000001)

In [50]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighbourhood_lat,
    neighbourhood_lng,
    LIMIT,
    radius
)

url

'https://api.foursquare.com/v2/venues/explore?client_id=Z5R30YV2S44XZOKFOBIH4NOBKDNK50UM5JEUGWC51TBFGWKY&client_secret=QSHHNLLS50BFLHHLQX2PZP3LZFAMCZ401MLBXDZUNR24FHVE&v=20210528&ll=43.6795626,-79.37752940000001&limit=100&radius=500'

In [55]:
# now we get the info from Foursquare and we put them in a dataframe

results = requests.get(url).json()

In [56]:
def get_category_type(row):
  try:
    categories_list = row['categories']
  
  except:
    categories_list = row['venue.categories']

  if len(categories_list) == 0:
    return None

  else:
    return categories_list[0]['name']

In [57]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten json into flat table

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category of each row fpr venues.categories
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

nearby_venues

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [58]:
print('{} venues were found by FourSquare'.format(nearby_venues.shape[0]))

4 venues were found by FourSquare


In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                latitudes=downtown_data['Latitude'],
                longitudes=downtown_data['Longitude'])

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Queen's Park


In [61]:
# let's check the shape of downtown venues dataframe

downtown_venues.shape

(1184, 7)

In [62]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,47,47,47,47,47,47
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",39,39,39,39,39,39
Central Bay Street,62,62,62,62,62,62
"Chinatown,Grange Park,Kensington Market",59,59,59,59,59,59
Christie,15,15,15,15,15,15
Church and Wellesley,66,66,66,66,66,66
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [63]:
print("There are {} unique catergories in Downtown Toronto.".format(len(downtown_venues['Venue Category'].unique())))

There are 191 unique catergories in Downtown Toronto.


### Now we start to analyze each neighbourhood

In [65]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues['Venue Category'], prefix='', prefix_sep='')

# add neighbourhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood']

# move Neighborhood column to the first column
cols = list(downtown_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
downtown_onehot = downtown_onehot.loc[:,cols]
downtown_onehot

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,...,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,"Cabbagetown,St. James Town",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,Queen's Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1180,Queen's Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1181,Queen's Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1182,Queen's Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,...,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.010000,0.020000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.00,0.000000,0.010000,0.020000,0.000000,...,0.000000,0.010000,0.010000,0.000000,0.030000,0.000000,0.000000,0.010000,0.050000,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.01,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.030000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.063830,0.000000,0.000000,0.00,0.021277,0.021277,0.000000,0.042553,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.021277,...,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.063830,0.00,0.000000,0.042553,0.000000,0.00,0.00,0.000000,0.000000,0.021277,0.00,0.021277,0.00,0.000000,0.021277,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.00,0.00,0.042553,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.058824,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.025641,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.025641,...,0.051282,0.000000,0.000000,0.000000,0.051282,0.000000,0.000000,0.000000,0.025641,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.025641,0.000000,0.000000,0.00,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.025641,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

### Display the top 10 venues for each neighborhood.

In [68]:
def return_most_common_venues(row, num_top_venues):
  row_categories = row.iloc[1:]
  row_categories_sorted = row_categories.sort_values(ascending=False)

  return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to placement of top venue
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
  try:
    columns.append('{}{} Most Common Venue.'.format(ind+1, indicators=ind))
    
  except:
    columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Sandwich Place,Gym,Clothing Store,Restaurant,Sushi Restaurant,Burrito Place,Asian Restaurant,Steakhouse
1,Berczy Park,Cocktail Bar,Bakery,Coffee Shop,Sandwich Place,Farmers Market,Vegetarian / Vegan Restaurant,Seafood Restaurant,Beer Bar,Beach,Park
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Boat or Ferry,Coffee Shop,Plane,Boutique,Rental Car Location,Harbor / Marina,Bar,Sculpture Garden
3,"Cabbagetown,St. James Town",Coffee Shop,Bakery,Pub,Restaurant,Café,Italian Restaurant,Pizza Place,Snack Place,Indian Restaurant,Japanese Restaurant
4,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Japanese Restaurant,Salad Place,Bank,Restaurant,Pizza Place


### Now we finally cluster the neighbours

In [70]:
# set number of clusters
k = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

#run k-Means
kmeans = KMeans(n_clusters=k, random_state=5).fit(downtown_grouped_clustering)

# check cluster labels generated for each row
kmeans.labels_[0:10]

array([1, 0, 4, 0, 1, 0, 3, 0, 1, 1], dtype=int32)

### Create a dataframe that contains the clusters labels as well as the top 10 venues for each neighborhood.

In [71]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [76]:
downtown_merged = downtown_data
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide,King,Richmond",Coffee Shop,Café,Sandwich Place,Gym,Clothing Store,Restaurant,Sushi Restaurant,Burrito Place,Asian Restaurant,Steakhouse
1,0,Berczy Park,Cocktail Bar,Bakery,Coffee Shop,Sandwich Place,Farmers Market,Vegetarian / Vegan Restaurant,Seafood Restaurant,Beer Bar,Beach,Park
2,4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Boat or Ferry,Coffee Shop,Plane,Boutique,Rental Car Location,Harbor / Marina,Bar,Sculpture Garden
3,0,"Cabbagetown,St. James Town",Coffee Shop,Bakery,Pub,Restaurant,Café,Italian Restaurant,Pizza Place,Snack Place,Indian Restaurant,Japanese Restaurant
4,1,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Japanese Restaurant,Salad Place,Bank,Restaurant,Pizza Place
5,0,"Chinatown,Grange Park,Kensington Market",Café,Coffee Shop,Vegetarian / Vegan Restaurant,Thai Restaurant,Burger Joint,Bar,Mexican Restaurant,Vietnamese Restaurant,Gaming Cafe,Food Court
6,3,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Nightclub,Restaurant,Athletics & Sports,Italian Restaurant,German Restaurant
7,0,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Coffee Shop,Gym,Fast Food Restaurant,Burrito Place,Mediterranean Restaurant,Indian Restaurant
8,1,"Commerce Court,Victoria Hotel",Coffee Shop,Sandwich Place,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Bank,Deli / Bodega,Cocktail Bar
9,1,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Restaurant,Pharmacy,Salad Place,Deli / Bodega,Japanese Restaurant


In [ ]:
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()